# PRIMO 2022 ML Homework 4<br>
### Estimating the issue priority in the bug tracker

Задача: предсказать, имеет ли проблема высокий приоритет, учитывая ее описание и метаданные.<br>
Данные: взяты с сайта youtrack.jetbrains.com.<br>
Оценка качества: F1 мера

Метрика оценки качества для соревнования: F1 мера для класса 1.<br>
<br>
**Формат файла с предсказаниями<br>**
Для каждой задачи в наборе test итоговый файл с предсказаниями должен содержать два значения: id (id задачи, взятый из столбца `id` в наборе данных test) и предсказание приоритета задачи (1, если приоритет высокий, или 0, если приоритет низкий).<br>
<br>
Файл должен иметь заголовок и следующий формат:<br>
<br>
id,is_high_priority<br>
0,1<br>
1,1<br>
2,1<br>

In [220]:
import pandas as pd
import numpy as np

import warnings
import json
import operator

from tqdm import tqdm

from sklearn.model_selection import train_test_split


from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.naive_bayes import MultinomialNB


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import classification_report
#np.set_printoptions(precision=3)
#pd.set_option('precision', 3)



In [221]:
test_data = pd.read_csv('test.csv', index_col='id')


In [222]:
test_data_ind = pd.read_csv('test.csv')

In [223]:
train_data = pd.read_csv('train.csv', index_col='id')

In [224]:
train_data.summary.fillna('', inplace=True)
train_data.description.fillna('', inplace=True)


In [225]:
test_data

,summary,description,reporter,created,customFields,links
id,,,,,,
25-2512303,WSL support for JVM development under Windows,As a Kotlin/Scala/Java developer on Windows I ...,"{""login"": ""joost.3"", ""$type"": ""User""}",1569949612582,"[{""value"": {""name"": ""Feature"", ""$type"": ""EnumB...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2513780,Mozilla's Fluent (*.ftl) syntax highlighting,What steps will reproduce the problem?\n\n1. O...,"{""login"": ""tobiaskybernium"", ""$type"": ""User""}",1570088574970,"[{""value"": {""name"": ""Feature"", ""$type"": ""EnumB...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2515293,Quick definition (Ctrl+Shift+I) called on Lomb...,What steps will reproduce the issue?\n1. Creat...,"{""login"": ""Sergei_Tsypanov"", ""$type"": ""User""}",1570190482389,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2514349,Checkboxes in search dialog not aligned to pix...,Ugly checkbox boundaries under windows 10 (per...,"{""login"": ""gamalik"", ""$type"": ""User""}",1570110634652,"[{""value"": {""name"": ""Cosmetics"", ""$type"": ""Enu...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2514337,HTTP client: Response Handler is not called wh...,**Problem**\n\nI can't use [response handler s...,"{""login"": ""iwansoft"", ""$type"": ""User""}",1570110085315,"[{""value"": {""name"": ""Usability Problem"", ""$typ...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
...,...,...,...,...,...,...
25-3224524,Incorrect Error Message Text Decoding of Java ...,Intellij uses incorrect text encoding for disp...,"{""login"": ""wh1099"", ""$type"": ""User""}",1616373544355,"[{""value"": {""name"": ""Configuration Problem"", ""...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-3224569,Code Style: Spacing for code braces not work,**What steps will reproduce the issue?**\n\n1....,"{""login"": ""blackj0221"", ""$type"": ""User""}",1616385044997,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-3224564,Diglog is hided by the taskbar when taskbar is...,Intellij IDEA 2020.3.3 \nwindows 10\n\n**What ...,"{""login"": ""myfirenze"", ""$type"": ""User""}",1616382740184,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."


mylist = []

for chunk in  pd.read_csv('train_2011_2012_2013.csv', sep=';', chunksize=20000):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)
del mylist

In [226]:
train_data1 = train_data.sample(frac = 1)

In [227]:
chunks= np.array_split(train_data1, 5)

In [228]:
train_data1 = chunks[0]

In [229]:
#chunks= np.array_split(train_data, 2)

In [230]:
#train_data1 = chunks[0]

In [231]:
test_data

,summary,description,reporter,created,customFields,links
id,,,,,,
25-2512303,WSL support for JVM development under Windows,As a Kotlin/Scala/Java developer on Windows I ...,"{""login"": ""joost.3"", ""$type"": ""User""}",1569949612582,"[{""value"": {""name"": ""Feature"", ""$type"": ""EnumB...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2513780,Mozilla's Fluent (*.ftl) syntax highlighting,What steps will reproduce the problem?\n\n1. O...,"{""login"": ""tobiaskybernium"", ""$type"": ""User""}",1570088574970,"[{""value"": {""name"": ""Feature"", ""$type"": ""EnumB...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2515293,Quick definition (Ctrl+Shift+I) called on Lomb...,What steps will reproduce the issue?\n1. Creat...,"{""login"": ""Sergei_Tsypanov"", ""$type"": ""User""}",1570190482389,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2514349,Checkboxes in search dialog not aligned to pix...,Ugly checkbox boundaries under windows 10 (per...,"{""login"": ""gamalik"", ""$type"": ""User""}",1570110634652,"[{""value"": {""name"": ""Cosmetics"", ""$type"": ""Enu...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-2514337,HTTP client: Response Handler is not called wh...,**Problem**\n\nI can't use [response handler s...,"{""login"": ""iwansoft"", ""$type"": ""User""}",1570110085315,"[{""value"": {""name"": ""Usability Problem"", ""$typ...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
...,...,...,...,...,...,...
25-3224524,Incorrect Error Message Text Decoding of Java ...,Intellij uses incorrect text encoding for disp...,"{""login"": ""wh1099"", ""$type"": ""User""}",1616373544355,"[{""value"": {""name"": ""Configuration Problem"", ""...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-3224569,Code Style: Spacing for code braces not work,**What steps will reproduce the issue?**\n\n1....,"{""login"": ""blackj0221"", ""$type"": ""User""}",1616385044997,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."
25-3224564,Diglog is hided by the taskbar when taskbar is...,Intellij IDEA 2020.3.3 \nwindows 10\n\n**What ...,"{""login"": ""myfirenze"", ""$type"": ""User""}",1616382740184,"[{""value"": {""name"": ""Bug"", ""$type"": ""EnumBundl...","[{""direction"": ""BOTH"", ""linkType"": {""directed""..."


In [232]:
#train_data1 = train_data.sample(frac = 1)
#chunks = np.array_split(train_data1, 10)
#train_data1 = chunks[0]

In [233]:
train_data1.summary.fillna('', inplace=True)
train_data1.description.fillna('', inplace=True)


In [234]:
test_data.summary.fillna('', inplace=True)
test_data.description.fillna('', inplace=True)

In [235]:
json_fields = pd.json_normalize(train_data1.customFields.map(json.loads).map(lambda x: {field['name']: field['value'] for field in x}))
json_fields.set_index(train_data1.index, inplace=True)
json_fields.fillna('',inplace=True)



In [236]:
json_fields_test = pd.json_normalize(test_data.customFields.map(json.loads).map(lambda x: {field['name']: field['value'] for field in x}))
json_fields_test.set_index(test_data.index, inplace=True)
json_fields.fillna('',inplace=True)



In [237]:
json_fields['Subsystem.name'].value_counts()

                                   4501
User Interface                     1382
Editor. Editing Text                791
Build. Maven                        699
Lang. Flash and Flex                687
                                   ... 
Version Control. VSS (archived)       1
Groovy. Inspections                   1
Frameworks. Micronaut                 1
Frameworks. Reactor                   1
JavaX. CDI                            1
Name: Subsystem.name, Length: 165, dtype: int64

In [238]:
json_columns = ['Type.name','State.name','Subsystem.name','Assignee.login']
train_data1 = train_data1.join(json_fields[json_columns], how='outer', lsuffix='_left')


train_data1['reporter_name'] = train_data1.reporter.map(json.loads).map(operator.itemgetter('login'))
train_data1['Year'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
train_data1['Month'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
train_data1['Day'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
train_data1['Hour'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)
prep_train_data = train_data1.drop(labels=['reporter', 'created', 'customFields', 'links'], axis=1)

In [239]:
json_columns = ['Type.name','State.name','Subsystem.name','Assignee.login']
train_data_test = train_data.join(json_fields[json_columns], how='outer', lsuffix='_left')


train_data_test['Year'] = train_data_test.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
train_data_test['Month'] = train_data_test.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
train_data_test['Day'] = train_data_test.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
train_data_test['Hour'] = train_data_test.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)
prep_train_data_test = train_data_test.drop(labels=['reporter', 'created', 'customFields', 'links'], axis=1)

In [240]:
train_data_test = prep_train_data_test.Year.isin([2019,2020,2021,2022])

In [241]:
prep_train_data_test[train_data_test]

,summary,description,is_high_priority,Type.name,State.name,Subsystem.name,Assignee.login,Year,Month,Day,Hour
id,,,,,,,,,,,
25-2214736,IntelliJ - Cannot Access Maven Repository Requ...,I have a local maven repository that requires ...,False,Bug,Duplicate,Build. Maven,vsoroka,2019,1,1,6
25-2214753,File choser dialog: error message communicated...,Trying to clone a new repository and I manuall...,False,NaN,NaN,NaN,NaN,2019,1,1,10
25-2214758,Cannot open Groovy console,When I trying to open Groovy Console it is sta...,False,NaN,NaN,NaN,NaN,2019,1,1,11
25-2214759,Classes annotated just with @SpringBootConfigu...,What steps will reproduce the issue?\n1. Creat...,False,Usability Problem,Duplicate,Frameworks. Spring,Konstantin.Aleev,2019,1,1,11
25-2214760,CTRL ALT S held down opens multiple settings d...,What steps will reproduce the issue?\n1. press...,False,NaN,NaN,NaN,NaN,2019,1,1,11
...,...,...,...,...,...,...,...,...,...,...,...
25-2509787,"Add option to ""Declaration can have final modi...","The majority of the time, having to declare me...",False,NaN,NaN,NaN,NaN,2019,9,29,16
25-2509792,Copying an editor tab focuses wrong editor,"![](image2.png)\n\nIn this situation, I want t...",False,NaN,NaN,NaN,NaN,2019,9,29,17
25-2509807,`Catch block may ignore exception` inspection ...,Steps to reproduce the issue:\n1. Declare a lo...,False,NaN,NaN,NaN,NaN,2019,9,29,17


In [242]:
json_columns_test = ['Type.name','State.name','Subsystem.name','Assignee.login']
test_data= test_data.join(json_fields_test[json_columns_test], how='outer', lsuffix='_left')


test_data['reporter_name'] = test_data.reporter.map(json.loads).map(operator.itemgetter('login'))
test_data['Year'] = test_data.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
test_data['Month'] = test_data.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
test_data['Day'] = test_data.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
test_data['Hour'] = test_data.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)
prep_test_data = test_data.drop(labels=['reporter', 'created', 'customFields', 'links'], axis=1)

In [243]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re
from sklearn.feature_selection import SelectFromModel

In [244]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

*сначала происходит лемматизация слов, и потом они отфильтровываются по условиям, так работать не должно*

In [245]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
       return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if re.fullmatch(r'[A-Za-z]{4,}', t) and t not in STOP_WORDS]

In [246]:
X_train_text = prep_train_data.summary.astype(str) +prep_train_data.description.astype(str)


In [247]:
X_train_text

id
25-356382     @Validateable constraints not shown properly i...
25-1292065    velocity good code red: calling function named...
25-74798      Provide live templateswe may probably start wi...
25-230560     Create subclass: import statement is not inser...
25-663004     IntelliJ doesn't recognize compile error when ...
                                    ...                        
25-82165      mxml: missing support for <mx:XML> tagThere ar...
25-1024417    Introduce collapsable/foldable buttons for qui...
25-1115221    Unable to save settings: Failed to save settin...
25-453730     Building for the adl emulator doesn't support ...
25-488347     Project view isn't updated on file creation/de...
Length: 21073, dtype: object

In [56]:
X_test_text = prep_test_data.summary.astype(str)# + " " + prep_test_data.description.astype(str)

In [57]:
X_test_text

id
25-2512303        WSL support for JVM development under Windows
25-2513780         Mozilla's Fluent (*.ftl) syntax highlighting
25-2515293    Quick definition (Ctrl+Shift+I) called on Lomb...
25-2514349    Checkboxes in search dialog not aligned to pix...
25-2514337    HTTP client: Response Handler is not called wh...
                                    ...                        
25-3224524    Incorrect Error Message Text Decoding of Java ...
25-3224569         Code Style: Spacing for code braces not work
25-3224564    Diglog is hided by the taskbar when taskbar is...
25-3224549    Debugger shall use "MethodParameters" attribut...
25-3224582    InteliJIdea Ultimate Keyboard shortcuts not wo...
Name: summary, Length: 18529, dtype: object

In [248]:
tfidf_vec_summary = TfidfVectorizer(min_df=.005, max_df=0.75,
                                    tokenizer=LemmaTokenizer(),
                             )
tfidf_vec_summary.fit(X_train_text)
train_text_prep = pd.DataFrame(tfidf_vec_summary.transform(X_train_text).todense(), index=X_train_text.index,
                               columns=tfidf_vec_summary.get_feature_names_out())
train_text_prep



c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,ability,able,abstract,accept,access,according,account,action,actionscript,active,...,work,workaround,worked,working,wrap,write,writing,wrong,xmlns,year
id,,,,,,,,,,,,,,,,,,,,,
25-356382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1292065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-74798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-230560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-663004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-82165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1024417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1115221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [249]:
train_text_prep

,ability,able,abstract,accept,access,according,account,action,actionscript,active,...,work,workaround,worked,working,wrap,write,writing,wrong,xmlns,year
id,,,,,,,,,,,,,,,,,,,,,
25-356382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1292065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-74798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-230560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-663004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-82165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1024417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1115221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [59]:
test_text_prep = pd.DataFrame(tfidf_vec_summary.transform(X_test_text).todense(), index=X_test_text.index,
                               columns=tfidf_vec_summary.get_feature_names_out())

In [60]:
test_text_prep

,ability,access,action,adding,allow,android,annotation,application,argument,artifact,...,user,value,variable,version,view,warning,window,work,working,wrong
id,,,,,,,,,,,,,,,,,,,,,
25-2512303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.643709,0.000000,0.000000,0.0
25-2513780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2515293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2514349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2514337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-3224524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.291703,0.000000,0.000000,0.0
25-3224569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.399193,0.000000,0.0
25-3224564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [252]:
prep_train_data

,Type.name,State.name,Subsystem.name,Assignee.login,Year,Month,Day,Hour
id,,,,,,,,
25-356382,Bug,Fixed,Groovy,sergey.evdokimov,2011,11,2,22
25-1292065,Bug,Fixed,Lang. Velocity,peter,2016,8,30,9
25-74798,Feature,Duplicate,Lang. Flash and Flex,nicity,2009,10,10,15
25-230560,Bug,Fixed,Lang. Flash and Flex,alexander.doroshko,2010,12,23,13
25-663004,Bug,Fixed,Java. Error Highlighting,anet,2014,2,10,22
...,...,...,...,...,...,...,...,...
25-82165,Bug,Fixed,Lang. Flash and Flex,alexander.doroshko,2009,12,8,21
25-1024417,Cosmetics,Submitted,,batrdmi,2015,9,28,13
25-1115221,Bug,Duplicate,,,2016,2,4,21


tfidf_vec_summary = TfidfVectorizer(min_df=.005, max_df=0.75,
                                    tokenizer=LemmaTokenizer(),
                             )

tfidf_vec_summary.fit_transform(X_train_text)


tfidf_vec_summary.fit(X_test_text)
test_text_prep = pd.DataFrame(tfidf_vec_summary.transform(X_test_text).todense(), index=X_test_text.index,
                               columns=tfidf_vec_summary.get_feature_names_out())
test_text_prep


In [264]:
Y = train_data1['is_high_priority']

In [266]:
Y

id
25-356382     False
25-1292065    False
25-74798      False
25-230560     False
25-663004     False
              ...  
25-82165      False
25-1024417    False
25-1115221    False
25-453730     False
25-488347     False
Name: is_high_priority, Length: 21073, dtype: bool

In [62]:
# X_train, X_test, y_train, y_test = train_test_split(train_text_prep, Y,
#                                                     stratify=Y,
#                                                     random_state=3,
#                                                     test_size=0.3,
#                                                     shuffle=True)

In [251]:
prep_train_data.drop(['description','summary','is_high_priority','reporter_name'], axis=1, inplace =True)

In [253]:


preprocessor = DataFrameMapper([
                                (['Type.name'], OneHotEncoder(sparse=False, handle_unknown='ignore', dtype=np.single)),
                                (['State.name'], OneHotEncoder(sparse=False, handle_unknown='ignore',dtype=np.single)),
                                (['Subsystem.name'], OneHotEncoder(sparse=False, handle_unknown='ignore',dtype=np.single)),
                                (['Assignee.login'],OneHotEncoder(sparse=False, handle_unknown='ignore',dtype=np.single)),
                                ('Year', None),
                                ('Month', None),
                                ('Day', None),
                                ('Hour', None),
                                ],
                               input_df=True,
                               df_out=True)



In [254]:
prep_train_data = preprocessor.fit_transform(prep_train_data)

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\de

In [69]:
prep_test_data = preprocessor.transform(prep_test_data)

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\utils\de

In [70]:

prep_test_data

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,Assignee.login_x0_yannc76,Assignee.login_x0_yole,Assignee.login_x0_yopox,Assignee.login_x0_yuriy.artamonov,Assignee.login_x0_zollycute,Assignee.login_x0_zolotov,Year,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
25-2512303,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2019,10,1,17
25-2513780,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2019,10,3,7
25-2515293,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,10,4,12
25-2514349,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,10,3,13
25-2514337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,10,3,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-3224524,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,3,22,0
25-3224569,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,3,22,3
25-3224564,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,3,22,3


In [255]:
prep_train_data

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,Assignee.login_x0_yannc76,Assignee.login_x0_yole,Assignee.login_x0_yopox,Assignee.login_x0_yuriy.artamonov,Assignee.login_x0_zollycute,Assignee.login_x0_zolotov,Year,Month,Day,Hour
id,,,,,,,,,,,,,,,,,,,,,
25-356382,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2011,11,2,22
25-1292065,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2016,8,30,9
25-74798,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2009,10,10,15
25-230560,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2010,12,23,13
25-663004,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,2,10,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-82165,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2009,12,8,21
25-1024417,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,28,13
25-1115221,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2016,2,4,21


,ability,access,action,adding,allow,android,annotation,application,argument,artifact,...,user,value,variable,version,view,warning,window,work,working,wrong
id,,,,,,,,,,,,,,,,,,,,,
25-2512303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.643709,0.000000,0.000000,0.0
25-2513780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2515293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2514349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2514337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-3224524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.291703,0.000000,0.000000,0.0
25-3224569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.399193,0.000000,0.0
25-3224564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [76]:
selector = SelectKBest(f_regression, k=2)


c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


[7.07181067e+01 7.88202727e+00 3.53944888e-01 ... 9.14677113e-02
 9.14677113e-02 4.57314347e-02]


array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [256]:
train_text_prep

,ability,able,abstract,accept,access,according,account,action,actionscript,active,...,work,workaround,worked,working,wrap,write,writing,wrong,xmlns,year
id,,,,,,,,,,,,,,,,,,,,,
25-356382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1292065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-74798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-230560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-663004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-82165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1024417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25-1115221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [25]:
#prep_train_data = prep_train_data.drop(['summary','description'], axis=1)

In [257]:
prep_train_data = prep_train_data.join(train_text_prep)

In [75]:
prep_test_data = prep_test_data.join(test_text_prep)

prep_test_data = prep_test_data.drop(['summary','description'], axis=1)
prep_test_data = prep_test_data.join(test_text_prep)

AttributeError: 'DataFrame' object has no attribute 'summary'

In [259]:
#pd.options.display.max_rows = 200

Y

id
25-356382     False
25-1292065    False
25-74798      False
25-230560     False
25-663004     False
              ...  
25-82165      False
25-1024417    False
25-1115221    False
25-453730     False
25-488347     False
Name: is_high_priority, Length: 21073, dtype: bool

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,user,value,variable,version,view,warning,window,work,working,wrong
id,,,,,,,,,,,,,,,,,,,,,
25-175178,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-2014506,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-1393869,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-1449354,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-1524206,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-567725,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-256240,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25-1118854,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


tfidf_vec_desc = TfidfVectorizer(min_df=.01, max_df=0.5,
                                   # token_pattern=r'[A-Za-z]{2,}',
                                    tokenizer=LemmaTokenizer(),
                                    stop_words=STOP_WORDS)
tfidf_vec_desc.fit(X_train_description)
X_train_description_sample = pd.DataFrame(tfidf_vec_desc.transform(X_train_description).todense(),
                                          columns=tfidf_vec_desc.get_feature_names_out())
X_train_description_sample

\# todo сократить размерность. Лемматизация, регулярные выражения поумнее, пороги для векторизации, объединить слова описания и заголовка?
отбор признаков по модели

In [258]:
X = prep_train_data


In [36]:
#dataFrameMapper_data = preprocessor.fit_transform(X)


In [37]:
#dataFrameMapper_data.to_csv('dataMapperData.csv')

In [38]:
#Y = Y.astype(np.int8)


,Year,Month,Day,Hour,ability,able,access,action,active,actual,...,Reporter_Павел_Швец,Reporter_Павлин_Краснопёров,Reporter_Петр_Королев,Reporter_Сергей.Камалтынов,Reporter_Станислав.Алёкминский,Reporter_Стас.Бицько,Reporter_Тимур_Шакуров,Reporter_Фагим.Садыков,Reporter_дэн.человек,Reporter_震_田
id,,,,,,,,,,,,,,,,,,,,,
25-1920680,2018,3,29,20,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-1018684,2015,9,20,20,0.0,0.0,0.0,0.130999,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-1188407,2016,4,20,19,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-787648,2014,11,12,15,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-499309,2013,1,1,19,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1784064,2017,11,28,10,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-302638,2011,4,19,16,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-679646,2014,3,20,10,0.0,0.0,0.0,0.156271,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [267]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.25,
                                                    shuffle=True)

id
25-152235     2010
25-2099191    2018
25-935185     2015
25-1275695    2016
25-201679     2010
              ... 
25-545162     2013
25-1071477    2015
25-566666     2013
25-327213     2011
25-367389     2011
Name: Year, Length: 15804, dtype: int64

In [153]:
prep_test_data

,Type.name_x0_Auto-reported Exception,Type.name_x0_Bug,Type.name_x0_Configuration Problem,Type.name_x0_Cosmetics,Type.name_x0_Exception,Type.name_x0_Feature,Type.name_x0_Meta Issue,Type.name_x0_Performance Problem,Type.name_x0_Security Problem,Type.name_x0_Support Request,...,user,value,variable,version,view,warning,window,work,working,wrong
id,,,,,,,,,,,,,,,,,,,,,
25-2512303,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.643709,0.000000,0.000000,0.0
25-2513780,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2515293,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2514349,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25-2514337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-3224524,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.291703,0.000000,0.000000,0.0
25-3224569,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.399193,0.000000,0.0
25-3224564,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [41]:
selector = SelectKBest(chi2)
selector.fit_transform(X_train, y_train)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [288]:
from sklearn.preprocessing import StandardScaler

In [306]:
LR = Pipeline([
        ('feature_selection', SelectFromModel(LinearSVC())),
        ('Standard', MinMaxScaler()),
        ('classification', LogisticRegression())
    ])
LR.fit(X_train, y_train)

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('feature_selection', SelectFromModel(estimator=LinearSVC())),
                ('Standard', MinMaxScaler()),
                ('classification', LogisticRegression())])

In [145]:
LR_test = Pipeline([
        ('feature_selection', SelectFromModel(LinearSVC())),
        ('classification', LogisticRegression())
    ])
LR_test.fit(X,Y)

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('feature_selection', SelectFromModel(estimator=LinearSVC())),
                ('classification', LogisticRegression())])

In [189]:
prediction_test = LR_test.predict_proba(prep_test_data)[:,1]

In [307]:
proba_train = LR.predict_proba(X_test)[:,1]

In [320]:
train_f1 = f1_score(y_true=y_test, y_pred=proba_train > 0.1)

In [321]:
train_f1

0.28746928746928746

In [190]:
prediction_test = prediction_test> 0.07

In [200]:
test_data_indexes = test_data_ind.id

In [201]:
test_data_indexes

0        25-2512303
1        25-2513780
2        25-2515293
3        25-2514349
4        25-2514337
            ...    
18524    25-3224524
18525    25-3224569
18526    25-3224564
18527    25-3224549
18528    25-3224582
Name: id, Length: 18529, dtype: object

In [203]:
dfInd = pd.DataFrame(test_data_indexes)

In [210]:
dfPred = pd.DataFrame(prediction_test, columns=['is_high_priority'])

In [211]:
test =  pd.concat([dfInd,dfPred], axis=1)

In [212]:
test

,id,is_high_priority
0,25-2512303,False
1,25-2513780,False
2,25-2515293,True
3,25-2514349,False
4,25-2514337,False
...,...,...
18524,25-3224524,False
18525,25-3224569,False
18526,25-3224564,False
18527,25-3224549,False


In [214]:
test.to_csv('predictions.csv',index=False)

In [192]:
unique, counts = np.unique(prediction_test, return_counts=True)

In [193]:
print(np.asarray((unique, counts)))

[[    0     1]
 [16922  1607]]


In [198]:
1607/16922

0.09496513414490013

In [130]:
prediction_train = LR.predict_proba(X_test)[:,1]

In [134]:
test_f1 = f1_score(y_true=y_test, y_pred=prediction_train > 0.07)

In [195]:
unique_train, counts_train = np.unique(y_test, return_counts=True)


In [196]:
print(np.asarray((unique_train, counts_train)))

[[   0    1]
 [5051  218]]


In [197]:
218/5051

0.04315977034250643

In [135]:
test_f1

0.2237885462555066

In [125]:
lr_test2 = LogisticRegression()
lr_test2.fit(X_train, y_train)
pred_test2 = lr_test2.predict_proba(X_test)[:,1]

c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [144]:
test2_f1 = f1_score(y_true=y_test, y_pred=pred_test2 > 0.09)
test2_f1

0.21339950372208435

In [44]:
param_grid = dict(threshold=[.05,.0023, 0.01, 0.07, 0.08], selector = [LogisticRegression(),KNeighborsClassifier(n_neighbors=2),ExtraTreesClassifier()] )

resultsProbaSelectModel = []
for params in tqdm(ParameterGrid(param_grid)):

    clf = Pipeline([
        ('feature_selection', SelectFromModel(LinearSVC())),
        ('classification', RandomForestClassifier())
    ])
    clf.fit(X_train, y_train)

    resultsProbaSelectModel.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=clf.predict_proba(X_train)[:, 1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=clf.predict_proba(X_test)[:,1] > params['threshold']),

    ))

  0%|          | 0/15 [00:00<?, ?it/s]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
  7%|▋         | 1/15 [01:02<14:33, 62.38s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 13%|█▎        | 2/15 [02:01<13:09, 60.70s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 20%|██        | 3/15 [03:02<12:05, 60.44s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 27%|██▋       | 4/15 [04:00<10:55, 59.64s/it]c:\users\w

In [45]:
pd.DataFrame(resultsProbaSelectModel).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
1,"{'selector': LogisticRegression(), 'threshold': 0.0023}",0.168175,0.097253
6,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.0023}",0.164791,0.098731
11,"{'selector': ExtraTreesClassifier(), 'threshold': 0.0023}",0.169545,0.101506
12,"{'selector': ExtraTreesClassifier(), 'threshold': 0.01}",0.289933,0.106065
2,"{'selector': LogisticRegression(), 'threshold': 0.01}",0.292485,0.109175
7,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.01}",0.292733,0.109321
13,"{'selector': ExtraTreesClassifier(), 'threshold': 0.07}",0.833575,0.122105
4,"{'selector': LogisticRegression(), 'threshold': 0.08}",0.857143,0.122402
5,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.05}",0.712284,0.123743
9,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.08}",0.855022,0.124294


In [43]:
pd.DataFrame(resultsProba).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
12,"{'selector': ExtraTreesClassifier(), 'threshold': 0.01}",0.085220,0.081572
2,"{'selector': LogisticRegression(), 'threshold': 0.01}",0.085178,0.081856
6,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.0023}",0.084972,0.081909
7,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.01}",0.085174,0.082127
1,"{'selector': LogisticRegression(), 'threshold': 0.0023}",0.084918,0.082228
11,"{'selector': ExtraTreesClassifier(), 'threshold': 0.0023}",0.084906,0.082438
13,"{'selector': ExtraTreesClassifier(), 'threshold': 0.07}",0.190345,0.103734
8,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.07}",0.189300,0.107216
10,"{'selector': ExtraTreesClassifier(), 'threshold': 0.05}",0.187883,0.109980
14,"{'selector': ExtraTreesClassifier(), 'threshold': 0.08}",0.191136,0.111801


In [ ]:
pred = clf.predict_proba(X_test)[:,1]

In [ ]:
threshold = 0.07
f1_score(y_true=y_test, y_pred= pred > threshold)

In [ ]:
# https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, MultinomialNB, ComplementNB

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = MinMaxScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

X_train_scaled

# todo надо бужет скэйлить!
from sklearn.ensemble import RandomForestRegressor
clf = SGDClassifier()

clf.fit(X_train_scaled,y_train)

In [ ]:
SVC_model = SVC()
# В KNN-модели нужно указать параметр n_neighbors
# Это число точек, на которое будет смотреть
# классификатор, чтобы определить, к какому классу принадлежит новая точка
KNN_model = KNeighborsClassifier(n_neighbors=5)
RFC_model = RandomForestClassifier()

SVC_model.fit(X_train, y_train)
KNN_model.fit(X_train, y_train)
RFC_model.fit(X_train, y_train)

SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)
RFC_model = RandomForestClassifier.predict_proba(X_test)[:,1]

In [ ]:
logReg = KNeighborsClassifier(n_neighbors=2)
logReg.fit(X_train_scaled, y_train)

In [ ]:
predictions = logReg.predict(X_test_scaled)

In [ ]:
proba = logReg.predict_proba(X_test_scaled)[:, 1]

In [ ]:
threshold = 0.03
f1_score(y_true=y_test, y_pred=proba > threshold)

In [ ]:
f1_score(y_true=y_test, y_pred=KNN_prediction)

In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05, .001,.0023, 0.003])

resultsProba = []
for params in tqdm(ParameterGrid(param_grid)):

  #  prediction = logReg.predict_proba(X_test_scaled)[:,1]
    resultsProba.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=logReg.predict_proba(X_train_scaled)[:, 1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=logReg.predict_proba(X_test_scaled)[:,1] > params['threshold']),

    ))


In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05, .001,.0023, 0.003])

resultsProba = []
for params in tqdm(ParameterGrid(param_grid)):

  #  prediction = logReg.predict_proba(X_test_scaled)[:,1]
    resultsProba.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=logReg.predict_proba(X_train_scaled)[:, 1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=logReg.predict_proba(X_test_scaled)[:,1] > params['threshold']),

    ))


In [ ]:
pd.DataFrame(resultsProba).sort_values('test_f1').style.bar(vmin=0, vmax=1)



prediction = clf.predict_proba(X_test)[:,1]
#print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_test))}')

prediction

score = clf.score(X_test, y_test)
score

In [ ]:
X_test

In [ ]:

minMaxTest_Y = prep_train_data.is_high_priority

In [ ]:
test_minMaxdf_X = prep_train_data.summary.astype(str) + " " + prep_train_data.description.astype(str)

In [ ]:
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(test_minMaxdf_X, minMaxTest_Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.25,
                                                    shuffle=True)

In [ ]:
X_train_minmax

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.005, 0.01,.05, 0.06], max_df=[.5,.6,.7,.8], classifier=[ComplementNB(),LogisticRegression()], threshold=[.15,.05, 0.1])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [ ('tfid', TfidfVectorizer(min_df= params['min_df'], max_df= params['max_df'],
                                    tokenizer=LemmaTokenizer(),
                             )),
                             ('to_dense', DenseTransformer()),
                             ('clf',  params['classifier'])
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train_minmax, y_train_minmax)


    probabilitiesText1 = pipe.predict_proba(X_train_minmax)[:, 1]
    probabilitiesText2 = pipe.predict_proba(X_test_minmax)[:, 1]

    results.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=probabilitiesText1 > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=probabilitiesText2 > params['threshold'])
    ))


In [ ]:
pd.DataFrame(results).sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:


#param_grid = dict(classifier=[ComplementNB(),LogisticRegression(),LinearRegression(), RandomForestRegressor()])




nb_pipeline = Pipeline(steps = [('scaler', MinMaxScaler()),
                                ('clf', ComplementNB())
                               ]
                       )

nb_pipeline.fit(X_train, y_train)

print (f'Train f1 = {f1_score(y_true=y_train, y_pred=nb_pipeline.predict(X_train))}')
print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_test))}')

In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05])

results = []
for params in tqdm(ParameterGrid(param_grid)):
    probabilities1 = nb_pipeline.predict_proba(X_train)[:, 1]
    probabilities2 = nb_pipeline.predict_proba(X_test)[:, 1]
    results.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=probabilities1 > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=probabilities2 > params['threshold'])
    ))



In [ ]:
pd.DataFrame(results).sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:


from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8], scaler=[MinMaxScaler()])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [
        ('scaler', params['scaler']),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train, y_train)
    train_preds = pipe.predict(X_test)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_test))
    ))


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [('tfidf', TfidfVectorizer(min_df=params['min_df'], max_df=params['max_df'],
                                                       token_pattern=r'[A-Za-z]{2,}',
                                                       stop_words=STOP_WORDS)),
                             ('to_dense', DenseTransformer()),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train, y_train)
    train_preds = pipe.predict(X_train)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_test))
    ))

In [ ]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:
train_data1

In [ ]:
probabilities = pipe.predict_proba(X)[:, 1]

In [ ]:
probabilities

In [ ]:
threshold = 0.6

In [ ]:
f1_score(y_true=y, y_pred=probabilities > threshold)